In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# 출력파서
- 모델이 생성한 텍스트를 구조화된 형식으로 변환하는 도구
- JSON, XML, CSV, Pydantic등
- 최근에는 모델들이 함수또는 도구호출을 지원하면서 이러한 작업을 자동으로 처리하는 경우가 늘고 있으므로,
- 가능하면 출력 파서 대신 함수/도구 호출을 사용하기를 권장함

|이름|입력유형|출력유형|설명|
|--|--|--|--|
|JSON|문자열\|메시지|JSON객체|지정된 JSON 객체 반환, Pydantic 모델 지원|
|XML|문자열\|메시지|딕셔너리|XML 태그의 딕셔너리 반환, XML 출력 필요시 사용|
|CSV|문자열\|메시지|문자열 목록(List[str])|쉼표로 구분된 값 목록 반환|
|OutputFixing|문자열\|메시지|-|다른 출력 파서의 오류 수정|
|RetryWithError|문자열\|메시지|-|출력 파서의 오류 수정 및 원본 지시 사항 전송|
|Pydantic|문자열\|메시지|Pydantic BaseModel|사용자 정의 Pydantic 모델 반환|
|YAML|문자열\|메시지|Pydantic BaseModel|YAML로 인코딩된 Pydantic 모델 반환|
|PandasDataFrame|문자열\|메시지|딕셔너리(dict)|Pandas DataFrame작업시 유용|
|Enum|문자열\|메시지|Enum|제공된 Enum 값 중 하나로 응답 구문 분석|
|Datetime|문자열\|메시지|datatime.datetime|응답을 datetime형식으로 구문 분석|
|Structured|문자열\|메시지|딕셔너리(Dict[str, str])|문자열 필드만 포함된 구조화된 정보 반환|

## 출력 파서의 세가지 주요 메서드
### 포맷 지침 가져오기
- 출력 파서는 언어 모델에게 응답을 어떤 형식으로 출력해야 하는지 알려주는 지침을 제공합니다.
- 출력 파서의 포맷 지침은 get_format_instructions() 메서드로 확인할 수 있습니다.

In [2]:
# 라이브러리 불러오기
from langchain_core.output_parsers import JsonOutputParser
#Json 출력 파서 불러오기
parser = JsonOutputParser()
instructions = parser.get_format_instructions()

print(instructions) # JSON 형식의 지침 출력

Return a JSON object.


### 파싱
- 출력파서의 parse() 메서드는 언어 모델의 응답을 받아 프로그래밍에서 사용하기 쉬운 형태(파이썬 딕셔너리)로 변환합니다.

In [3]:
ai_response = '{"이름":"김철수", "나이": 30}'
parsed_response = parser.parse(ai_response)
print(parsed_response)

{'이름': '김철수', '나이': 30}


### 프롬프트와 함께 파싱
- 출력 파서의 parse_with_prompt() 메서드를 활용하면, 언어 모델의 응답과 함께 해당 질문(프롬프트)까지 받아서 데이터를 분석할 수 있습니다.
- 주로 AI 응답에 오류가 발생했을 때, 문제를 수정하거나 다시 시도할 때 유용합니다.
- RetryWithErrorOutputParser를 이용하면, 출력 파싱과정에서 발생하는 오류를 관리합니다.
- 원래의 프롬프트와 AI응답, 발생한 오류 메시지를 함께 전달하여 AI가 더 나은 수정 제안을 할 수 있도록 유도합니다.

In [4]:
from langchain.output_parsers import RetryWithErrorOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

# 파서 설정
parser = RetryWithErrorOutputParser.from_llm(parser=JsonOutputParser(), llm=ChatOpenAI())

- RetryWithErrorOutputParser : 오류 발생시 재시도 기능
- JsonOutputParser : 내부 파서로 사용하여 JSON 형식으로 파싱 담당
- ChatOpenAI : LLM모델을 활용하여 오류 수정을 요청할 수 있도록 함

In [ ]:
question = "가장 큰 대륙은?"
ai_response = "아시아입니다." # JSON 형식이 아닌 잘못된 대답
# ai_response = '{"answer": "아시아"}'

try:
    result = parser.parse_with_prompt(ai_response, question)
    print(result) # JSON 형식으로 변환된 결과 출력
except Exception as e:
    print(f"오류 발생: {e}")
    # 여기서 AI에게 다시 질문할 수 있습니다.

{'answer': '아시아'}


- AI가 반환할 가상의 응답을 설정합니다. 의도적으로 JSON이 아닌 잘못된 형식으로 설정되어 있습ㄴ디ㅏ.
- 오류가 발생하지 않도록 수정하려면

```python
ai_response = '{"answer": "아시아"}'
```

## PydanticOutputParser
- PydanticOutputParser는 랭체인에서 제공하는 도구, LLM이 생성한 텍스트를 구조화된 데이터로 변환하는데 사용됩니다.
- Pydantic과 함께 사용하여 AI가 생성한 자유 형식의 텍스트를 개발자가 설정한 데이터 구조에 맞춰 자동으로 변환하고 유효성을 확인합니다.
- AI가 형식을 따르지 않을 경우 오류가 발생할 수 있으므로 프롬프트 설계가 중요합니다

In [2]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, model_validator

- PydanticOutputParser : AI의 출력을 Pydantic 모델에 맞게 구조화된 데이터로 변환하며, 일관된 형식과 데이터 검증을 제공합니다.
- PromptTemplate : 프롬프트를 동적으로 생성하여 AI에게 전달할 질문과 출력 형식 지침을 설정합니다. 변수를 포함하여 다양한 입력을 처리할 수 있습니다.
- ChatOpenAI : 오픈AI의 GPT-4o 모델을 랭체인을 통해 사용할 수 있도록 지원하며, AI 모델에 질문을 던지고 응답을 얻는 역할을 합니다.
- pydantic : Python에서 데이터 검증과 모델링을 위한 라이브러리로 BaseModel과 Field를 사용하여 데이터 구조를 정의하고, model_validator를 통해 입력 데이터를 검증할 수 있습니다.

In [21]:
# OpenAI 모델 설정
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [22]:
# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")
    # Pydantic을 사용한 사용자 정의 검증 로직
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup", "")
        if not setup.endswith("?"):
            raise ValueError("잘못된 질문 형식입니다! 질문은 '?'로 끝나야 합니다.")
        return values

Pydantic을 사용하여 FinancialAdvice라는 클래스를 정의합니다.
AI 모델이 제공하는 데이터를 구조화된 형태로 저장하기 위한 데이터 모델을 만듭니다.

- BaseModel: Pydantic의 기본 모델 클래스로 데이터를 구조화하고 검증하는데 사용되며
- Field: 각 필드에 대한 설명과 유효성 검증 정보를 추가하는 데 사용됩니다.
- setup 필드: 금융 조언 상황을 나타내는 질문
- advice 필드: 해당 질문에 대한 구체적인 금융 조언 답변을 저장하는 역할을 합니다.
- model_validator 데코레이터: 질문 형식이 올바른지 검증하는 로직. question_ends_with_question_mark 클래스 메서드에서 처리됩니다.
  - AI가 생성한 질문이 물음표로(?) 끝나는지 확인

In [23]:
# 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해주세요.\n{format_instructions}\n질문: {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
#언어 모델을 사용해 데이터 구조를 채우도록 프롬프트와 모델 설정
chain = prompt | model | parser

parser.get_format_instructions는 AI에게 답변 형식을 지정하는 지침을 제공하여, AI가 지정된 형식에 맞는 답변을 생성할 수 있게 도와줍니다.

In [25]:
# 체인 실행 및 결과 출력
try:
    result = chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있게 질문과 답변을 해주세요."})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")


setup='부동산 투자에 대한 조언을 원합니다. 어떤 요소를 고려해야 할까요?' advice='부동산 투자 시 위치, 시장 동향, 자금 조달 방법, 임대 수익률, 세금 혜택 등을 고려해야 합니다. 또한, 장기적인 투자 계획을 세우고, 전문가의 조언을 받는 것이 중요합니다.'


## SimpleJsonOutputParser
- 랭체인 라이브러리에서 제공하는 출력 파서중 하나로 JSON 형식으로 파싱하는데 사용됩니다.
- JSON형식이 필요하지만 Pydantic 모델과 같은 복잡한 구조가 필요하지 않을때 특히 유용합니다.
- 실시간 처리와 스트리밍 기능을 지원하여 효율적으로 처리할 수 있습니다.
- 언어 모델이 항상 완벽한 JSON형식을 생성하는 것은 아니므로 오류처리가 중요합니다.

In [26]:
from langchain.output_parsers.json import SimpleJsonOutputParser
# JSON 형식의 응답을 생성하는 프롬프트 템플릿 설정
json_prompt = PromptTemplate.from_template(
    "다음 질문에 대한 답변이 포함된 JSON 객체를 반환하십시오: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

SimpleJsonOutputParser의 가장 큰 장점은 스트리밍 방식으로 JSON 출력을 처리할 수 있다는 점입니다.

In [27]:
# 스트리밍 예시: 질문에 대한 답변이 점진적으로 구문 분석됨
list(json_chain.stream({"question": "비트코인에 대한 짧은 한 문장 설명."}))

[{},
 {'description': ''},
 {'description': '비'},
 {'description': '비트'},
 {'description': '비트코'},
 {'description': '비트코인은'},
 {'description': '비트코인은 중앙'},
 {'description': '비트코인은 중앙은행'},
 {'description': '비트코인은 중앙은행이나'},
 {'description': '비트코인은 중앙은행이나 정부'},
 {'description': '비트코인은 중앙은행이나 정부의'},
 {'description': '비트코인은 중앙은행이나 정부의 개'},
 {'description': '비트코인은 중앙은행이나 정부의 개입'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가 가능한'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가 가능한 디'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가 가능한 디지털'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가 가능한 디지털 통'},
 {'descrip

## JSONOutputParser
- 랭체인에서 제공하는 또 다른 유용한 출력 파서로, JSON 형식의 출력을 요청하고 파싱하는 데 사용됩니다.
- Pydantic과 함께 사용하여 구조화된 데이터를 처리할 수 있습니다.
- AI모델이 생성한 자유 형식의 텍스트를 JSON구조로 명확히 변환할 수 있습니다.

In [28]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 오픈AI 모델 설정
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")

In [29]:
# JSON 출력 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = JsonOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해주세요.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

JsonOutputParser를 통해 Pydantic 모델에 맞는 출력 형식을 설정합니다.

In [30]:
#체인 구성: 프롬프트 -> 모델 -> 파서
chain = prompt | model | parser
# 체인 실행
chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있게 질문과 답변을 해주세요."})

{'setup': '부동산 투자에 대한 초기 자본금이 부족한 상황에서 어떻게 자금을 마련할 수 있을까요?',
 'advice': '부동산 투자를 위해 자금을 마련하는 방법으로는 여러 가지가 있습니다. 첫째, 저축을 통해 자본금을 늘리는 방법이 있습니다. 둘째, 가족이나 친구에게 자금을 빌리는 것도 고려해볼 수 있습니다. 셋째, 은행에서 주택담보대출이나 개인대출을 이용하는 방법도 있습니다. 마지막으로, 크라우드 펀딩 플랫폼을 통해 소액 투자자들로부터 자금을 모으는 방법도 있습니다. 각 방법의 장단점을 잘 비교하고, 자신의 상황에 맞는 최선의 선택을 하시기 바랍니다.'}

- 여러가지 파서 참고
- https://python.langchain.com/docs/how_to/#output-parsers